In [85]:
import traci
import traci.constants as tc
import os
import sys
import time

In [86]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

In [87]:
sumo_binary = r'C:\Program Files (x86)\Eclipse\Sumo\bin\sumo-gui'
sumo_cmd = [sumo_binary, '-c', 'traffic.sumocfg']

In [121]:
traci.close()

In [122]:
traci.start(sumo_cmd)

(20, 'SUMO 1.15.0')

In [123]:
def group_lanes_flow(lanes):
    flow_lanes = {}
    for lane in lanes:
        flow_id_length = 2
        if lane[0] == '-':
            flow_id_length = 3
        
        if not lane[:flow_id_length] in flow_lanes:
            flow_lanes[lane[:flow_id_length]] = {
                'lanes': [lane]
            }
        else:
            flow_lanes[lane[:flow_id_length]]['lanes'].append(lane)

    return flow_lanes

In [124]:
def get_vehicle_list(flow):
    flow_vehicle_list = []
    for lane in flow['lanes']:
        vehicle_list = traci.lane.getLastStepVehicleIDs(lane)
        flow_vehicle_list.extend(vehicle_list)
        
    return flow_vehicle_list

In [125]:
def count_vehicle_per_flow(flow):
    total_vehicle = 0
    for lane in flow['lanes']:
        total_vehicle += traci.lane.getLastStepVehicleNumber(lane)
        
    return total_vehicle

In [127]:
# input: {'vehicle_id': waiting_time(int)}
def calc_vehicle_waiting_time(vehicle_list_dict, vehicle_on_simulation):
    for vehicle in vehicle_list_dict:
        if vehicle in vehicle_on_simulation:
            waiting_time = traci.vehicle.getAccumulatedWaitingTime(vehicle)
            vehicle_list_dict[vehicle] = max(vehicle_list_dict[vehicle], waiting_time)
            
    return vehicle_list_dict

In [128]:
def calc_flow_stats(flow):
#     avg_vehicle_per_minute = len(flow['vehicle_list'])
    total_vehicle = len(flow['vehicle_list'])
    total_waiting_time = 0
    for vehicle in flow['vehicle_list']:
        total_waiting_time += flow['vehicle_list'][vehicle]
    avg_vehicle_waiting_time = total_waiting_time/total_vehicle
    
    return {
        'total_vehicle': total_vehicle,
        'total_waiting_time': total_waiting_time,
        'avg_vehicle_waiting_time': avg_vehicle_waiting_time
    }

## read traffic condition

In [75]:
tf_id = traci.trafficlight.getIDList()[0]
lanes = traci.trafficlight.getControlledLanes(tf_id)
# remove duplicates
lanes = list(dict.fromkeys(lanes))
# group lanes into same flow
flow_lanes = group_lanes_flow(lanes)
for flow in flow_lanes:
    # create vehicle_list key if not exist yet
    if 'vehicle_list' not in flow_lanes[flow]:
        flow_lanes[flow]['vehicle_list'] = {}

    # identify list of vehicles currently (current step) on the lanes
    new_vehicle_list = get_vehicle_list(flow_lanes[flow])
    # identify list of vehicles that already in the dict
    exisiting_vehicle_list = list(flow_lanes[flow]['vehicle_list'].keys())
    # new vehicle to add
    new_vehicle_to_add = list(set(new_vehicle_list).difference(set(exisiting_vehicle_list)))
    # add new vehicles (that does not exist in the dict yet) into the dict with initial waiting time value of 0
    flow_lanes[flow]['vehicle_list'] = dict(list(flow_lanes[flow]['vehicle_list'].items()) + list(dict.fromkeys(new_vehicle_to_add, 0.0).items())) 
    # calculate waiting time for all vehicle in list
    vehicle_on_simulation = traci.vehicle.getIDList()
    flow_lanes[flow]['vehicle_list'] = calc_vehicle_waiting_time(flow_lanes[flow]['vehicle_list'], vehicle_on_simulation)

## Run the simulation

In [ ]:
# run without value retrieval
step = 0
while step < 100:
    traci.simulationStep()
    lanes = traci.trafficlight.getControlledLanes('J3')
    step += 1
    time.sleep(0.0)

In [130]:
# run this with value retrieval!
tf_id = traci.trafficlight.getIDList()[0]
lanes = traci.trafficlight.getControlledLanes(tf_id)
# remove duplicates
lanes = list(dict.fromkeys(lanes))
# group lanes into same flow
flow_lanes = group_lanes_flow(lanes)
step = 0
while step < 1000:
    traci.simulationStep()
    lanes = traci.trafficlight.getControlledLanes(tf_id)
    step += 1
    
    for flow in flow_lanes:
        # create vehicle_list key if not exist yet
        if 'vehicle_list' not in flow_lanes[flow]:
            flow_lanes[flow]['vehicle_list'] = {}

        # identify list of vehicles currently (current step) on the lanes
        new_vehicle_list = get_vehicle_list(flow_lanes[flow])
        # identify list of vehicles that already in the dict
        exisiting_vehicle_list = list(flow_lanes[flow]['vehicle_list'].keys())
        # new vehicle to add
        new_vehicle_to_add = list(set(new_vehicle_list).difference(set(exisiting_vehicle_list)))
        # add new vehicles (that does not exist in the dict yet) into the dict with initial waiting time value of 0
        flow_lanes[flow]['vehicle_list'] = dict(list(flow_lanes[flow]['vehicle_list'].items()) + list(dict.fromkeys(new_vehicle_to_add, 0.0).items())) 
        # calculate waiting time for all vehicle in list
        vehicle_on_simulation = traci.vehicle.getIDList()
        flow_lanes[flow]['vehicle_list'] = calc_vehicle_waiting_time(flow_lanes[flow]['vehicle_list'], vehicle_on_simulation)
    
#     if step % 10 == 0:
#         print(flow_lanes['E0'])
    time.sleep(0.01)

for flow in flow_lanes:
    stats = calc_flow_stats(flow_lanes[flow])
    flow_lanes[flow]['stats'] = stats

In [139]:
flow_lanes['E1']['stats']

{'total_vehicle': 335,
 'total_waiting_time': 4022.0,
 'avg_vehicle_waiting_time': 12.00597014925373}

In [111]:
traci.close()

In [104]:
traci.trafficlight.getRedYellowGreenState(tf_id)

'rrrrGGGGgrrrrGGGGg'

In [112]:
traci.trafficlight.getAllProgramLogics(tf_id)[0]

Logic(programID='0', type=0, currentPhaseIndex=0, phases=(Phase(duration=42.0, state='rrrrGGGGgrrrrGGGGg', minDur=42.0, maxDur=42.0, next=()), Phase(duration=3.0, state='rrrryyyyyrrrryyyyy', minDur=3.0, maxDur=3.0, next=()), Phase(duration=42.0, state='GGGgrrrrrGGGgrrrrr', minDur=42.0, maxDur=42.0, next=()), Phase(duration=3.0, state='yyyyrrrrryyyyrrrrr', minDur=3.0, maxDur=3.0, next=())), subParameter={})

In [116]:
print(traci.trafficlight.getAllProgramLogics(tf_id)[0].phases[0])
print()
print(traci.trafficlight.getAllProgramLogics(tf_id)[0].phases[1])
print()
print(traci.trafficlight.getAllProgramLogics(tf_id)[0].phases[2])
print()
print(traci.trafficlight.getAllProgramLogics(tf_id)[0].phases[3])
print()

Phase(duration=42.0, state='rrrrGGGGgrrrrGGGGg', minDur=42.0, maxDur=42.0, next=())

Phase(duration=3.0, state='rrrryyyyyrrrryyyyy', minDur=3.0, maxDur=3.0, next=())

Phase(duration=42.0, state='GGGgrrrrrGGGgrrrrr', minDur=42.0, maxDur=42.0, next=())

Phase(duration=3.0, state='yyyyrrrrryyyyrrrrr', minDur=3.0, maxDur=3.0, next=())



# Archive

## List of element in Traci
- gui
- lane
- poi
- simulation
- trafficlight
- vehicletype
- edge
- inductionloop
- junction
- multientryexit
- polygon
- route
- person
- vehicle

In [16]:
traci.trafficlight.getIDList()

('J3',)

In [7]:
junctionID = traci.junction.getIDList()[0]
junctionID

':J3_18_0'

In [13]:
traci.route

https://github.com/LucasAlegre/sumo-rl/blob/master/sumo_rl/environment/traffic_signal.py

In [36]:
# minimum gap between cars set by sumo
MIN_GAP = 2.5

In [32]:
# get list of lanes controlled by the traffic light (incoming lanes)
lanes = traci.trafficlight.getControlledLanes('J3')
lanes

('-E4_0',
 '-E4_0',
 '-E4_1',
 '-E4_1',
 'E2_0',
 'E2_0',
 'E2_1',
 'E2_2',
 'E2_2',
 '-E3_0',
 '-E3_0',
 '-E3_1',
 '-E3_1',
 'E1_0',
 'E1_0',
 'E1_1',
 'E1_2',
 'E1_2')

In [33]:
# get lane length
[traci.lane.getLength(lane) for lane in lanes]

[86.4,
 86.4,
 86.4,
 86.4,
 189.6,
 189.6,
 189.6,
 189.6,
 189.6,
 86.4,
 86.4,
 86.4,
 86.4,
 189.6,
 189.6,
 189.6,
 189.6,
 189.6]

In [19]:
# get number of vehicles stop for the red light in the lane
[traci.lane.getLastStepHaltingNumber(lane) for lane in lanes]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [35]:
# get number of vehicles in the lane
[traci.lane.getLastStepVehicleNumber(lane) for lane in lanes]

[5, 5, 2, 2, 8, 8, 2, 2, 2, 5, 5, 2, 2, 8, 8, 3, 2, 2]

In [38]:
# get individual vehicle id
traci.lane.getLastStepVehicleIDs(lanes[0])

('f_2.34', 'f_2.33', 'f_2.32', 'f_2.30', 'f_2.28')

In [49]:
# get individual vehicle waiting time, 0 if the car is on the move
traci.vehicle.getAccumulatedWaitingTime('f_0.31')

0.0

In [37]:
# i think it's the car length
[traci.lane.getLastStepLength(lane) for lane in lanes]

[5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0]

In [43]:
# get waiting time per lane
def get_accumulated_waiting_time_per_lane(lane_id):
    vehicle_list = traci.lane.getLastStepVehicleIDs(lane_id)
    lane_waiting_time = 0.0
    for vehicle in vehicle_list:
        vehicle_lane = traci.vehicle.getLaneID(vehicle)
        vehicle_waiting_time = traci.vehicle.getAccumulatedWaitingTime(vehicle)
        
        lane_waiting_time += vehicle_waiting_time
        
    return lane_waiting_time

In [48]:
for lane in lanes:
    waiting_time = get_accumulated_waiting_time_per_lane(lane)
    print(f'{lane}: {waiting_time}')

-E4_0: 9.0
-E4_0: 9.0
-E4_1: 4.0
-E4_1: 4.0
E2_0: 27.0
E2_0: 27.0
E2_1: 10.0
E2_2: 0.0
E2_2: 0.0
-E3_0: 11.0
-E3_0: 11.0
-E3_1: 4.0
-E3_1: 4.0
E1_0: 23.0
E1_0: 23.0
E1_1: 12.0
E1_2: 3.0
E1_2: 3.0


In [ ]:
lanes_queue = [self.sumo.lane.getLastStepHaltingNumber(lane) / (self.lanes_lenght[lane] / (self.MIN_GAP + self.sumo.lane.getLastStepLength(lane))) for lane in self.lanes]